In [105]:
import pandas as pd

In [106]:
df = pd.read_excel('Data/Unlinked Metadata 0625.xlsx')
df.head()

,Subject ID,Patient Consented,Sample Type_#,CGA,DOL,Current Weight,Current Height,Current HC,Scavenged/Fresh?,MBM/DMB?,HMF Y/N?,TPN Y/N?,Iron Y/N?,Linked?,# Aliquots,Additional Comments
0,NB00012,N,NB00012_M_1,28.4,27,1430,32.0,26.0,Scavenged,MBM + DBM,Y,NaN,NaN,N,5 (~0.8),Residual from Milk Prep Room & B2 Full
1,NB00003,Y,NB00003_M_1,29.0,22,995,35.7,23.7,Scavenged,DBM,Y,N,Y,N,2,Scavenged Feeding Tube
2,NB00003,Y,NB00003_M_2,29.0,22,1005,36.0,24.4,Scavenged,DBM,Y,N,Y,N,1,Scavenged Feeding Tube
3,NB00003,Y,NB00003_M_3,29.3,25,1065,36.0,24.3,Scavenged,DBM,Y,N,Y,N,1,Scavenged Feeding Tube
4,NB00354,N,NB00354_M_1,29.4,12,1005,34.0,34.5,Scavenged,MBM,Y,Y,NaN,N,7,Residual from Milk Prep Room


In [107]:
AC = pd.read_excel('Data/NeoBank Unlinked AC.xlsx', sheet_name='relative')
AC.head()

,sample ID,inj vol,2'FL,DFLAC,3'SL,6'SL,LNT,LNnT,LNFPI,LNFPII,LNFPIII,LSTc,DFLNT,DSLNT,DFLNH,FDSLNH,DSLNH
0,NB00003_M_27,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NB00003_M_26,15,0.149068,0.025539,0.036240,0.048252,0.032744,0.015477,0.037367,0.029173,0.014387,0.029336,0.024442,0.024050,0.014162,0.024403,0.014275
2,NB00010_M_4,15,3.202351,0.211063,0.202066,0.793208,1.288569,0.099609,0.896822,0.838056,0.026299,0.198385,0.816358,0.293375,0.064694,0.116699,0.158850
3,NB00004_M_4,15,4.533413,0.015644,0.109186,0.516448,0.978176,0.235237,2.135244,0.025100,0.028135,0.273544,0.066877,0.159376,0.057803,0.033015,0.069969
4,NB00014_M_1,15,6.616323,0.997031,0.286334,0.743593,1.071591,0.337355,2.218069,0.367415,0.051010,0.659550,0.795395,0.261531,0.038317,0.048243,0.148319


In [108]:
df = df.rename(columns={"Sample Type_#": "sample_unique_id"})
AC = AC.rename(columns={"sample ID": "sample_unique_id"})


In [109]:
df['sample_unique_id'] = df['sample_unique_id'].str.strip()
AC['sample_unique_id'] = AC['sample_unique_id'].str.strip()

In [110]:
AC

,sample_unique_id,inj vol,2'FL,DFLAC,3'SL,6'SL,LNT,LNnT,LNFPI,LNFPII,LNFPIII,LSTc,DFLNT,DSLNT,DFLNH,FDSLNH,DSLNH
0,NB00003_M_27,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NB00003_M_26,15,0.149068,0.025539,0.036240,0.048252,0.032744,0.015477,0.037367,0.029173,0.014387,0.029336,0.024442,0.024050,0.014162,0.024403,0.014275
2,NB00010_M_4,15,3.202351,0.211063,0.202066,0.793208,1.288569,0.099609,0.896822,0.838056,0.026299,0.198385,0.816358,0.293375,0.064694,0.116699,0.158850
3,NB00004_M_4,15,4.533413,0.015644,0.109186,0.516448,0.978176,0.235237,2.135244,0.025100,0.028135,0.273544,0.066877,0.159376,0.057803,0.033015,0.069969
4,NB00014_M_1,15,6.616323,0.997031,0.286334,0.743593,1.071591,0.337355,2.218069,0.367415,0.051010,0.659550,0.795395,0.261531,0.038317,0.048243,0.148319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,NB00380_M_5,15,0.304270,0.011897,0.263096,0.820189,1.305836,0.335492,0.132126,1.380412,0.017233,0.378374,0.178397,0.149128,0.099998,0.246744,0.249289
131,NB00438_M_3,15,4.197056,0.353557,0.210112,0.510462,0.837567,0.176720,1.504402,0.391415,0.019311,0.300946,0.587013,0.103855,0.043828,0.038037,0.116563
132,NB00457_M_1,15,7.213692,0.555859,0.342633,1.165302,2.305625,0.246715,3.471317,0.730289,0.109193,0.914475,1.595734,0.311655,0.048459,0.029148,0.162687
133,NB00468_M_1,15,3.254378,0.394317,0.246220,0.331024,0.403736,0.098364,0.497896,0.573353,0.019541,0.130040,0.511980,0.100318,0.066628,0.068355,0.069158


In [111]:
merged = pd.merge(df, AC, on="sample_unique_id", how="inner")


In [112]:
merged

,Subject ID,Patient Consented,sample_unique_id,CGA,DOL,Current Weight,Current Height,Current HC,Scavenged/Fresh?,MBM/DMB?,...,LNnT,LNFPI,LNFPII,LNFPIII,LSTc,DFLNT,DSLNT,DFLNH,FDSLNH,DSLNH
0,NB00012,N,NB00012_M_1,28.4,27,1430,32.0,26.0,Scavenged,MBM + DBM,...,0.089623,0.470256,1.205369,0.014793,0.239572,0.281999,0.173245,0.215138,0.348744,0.384665
1,NB00003,Y,NB00003_M_1,29.0,22,995,35.7,23.7,Scavenged,DBM,...,0.054806,0.337448,0.352980,0.011570,0.075870,0.375191,0.066067,0.044498,0.038670,0.034095
2,NB00003,Y,NB00003_M_2,29.0,22,1005,36.0,24.4,Scavenged,DBM,...,0.113728,0.508929,0.384073,0.009342,0.082319,0.296118,0.063142,0.073261,0.057761,0.057419
3,NB00003,Y,NB00003_M_3,29.3,25,1065,36.0,24.3,Scavenged,DBM,...,0.078861,0.579639,0.381617,0.012011,0.080428,0.181844,0.061159,0.052973,0.090722,0.052794
4,NB00354,N,NB00354_M_1,29.4,12,1005,34.0,34.5,Scavenged,MBM,...,0.049258,0.108794,1.039603,0.024323,0.258750,0.138556,0.164200,0.203099,0.342770,0.364438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,NB00007,Y,NB00007_M_2,38.4,12,2190,44.0,32.5,Scavenged,MBM,...,0.197148,1.031503,0.691480,0.038184,0.353092,0.681753,0.194094,0.063247,0.092453,0.154003
124,NB00007,Y,NB00007_M_3,39.2,17,2340,44.0,33.0,Scavenged,MBM,...,0.093871,0.826393,0.565923,0.023937,0.160066,0.732057,0.248158,0.140775,0.036514,0.117773
125,NB00007,Y,NB00007_M_4,39.3,18,2380,44.0,33.0,Scavenged,MBM,...,0.062626,0.250724,1.749928,0.062896,0.362996,0.159926,0.227344,0.067247,0.204275,0.226578
126,NB00379,Y,NB00379_M_1,39.6,98,3073,45.0,33.2,Scavenged,MBM,...,0.074441,1.492769,0.211561,0.009555,0.160130,0.005741,0.066606,0.061999,0.013559,0.146605


In [113]:
merged.columns

Index(['Subject ID', 'Patient Consented', 'sample_unique_id', 'CGA', 'DOL ',
       'Current Weight', 'Current Height', 'Current HC', 'Scavenged/Fresh?',
       'MBM/DMB?', 'HMF Y/N?', 'TPN Y/N?', 'Iron Y/N? ', 'Linked? ',
       '# Aliquots ', 'Additional Comments', 'inj vol', '2'FL', 'DFLAC',
       '3'SL', '6'SL', 'LNT', 'LNnT', 'LNFPI', 'LNFPII', 'LNFPIII', 'LSTc',
       'DFLNT', 'DSLNT', 'DFLNH', 'FDSLNH', 'DSLNH'],
      dtype='object')

* seeing 122 rows in the inner join - aka 122 samples that exist in both datasets
* seeing 135 rows in the left join - aka 135 samples that exist in the df dataset, fills unmatched HMO data with NaN

In [114]:
merged_2 = pd.merge(df, AC, on="sample_unique_id", how="left")
merged_2

,Subject ID,Patient Consented,sample_unique_id,CGA,DOL,Current Weight,Current Height,Current HC,Scavenged/Fresh?,MBM/DMB?,...,LNnT,LNFPI,LNFPII,LNFPIII,LSTc,DFLNT,DSLNT,DFLNH,FDSLNH,DSLNH
0,NB00012,N,NB00012_M_1,28.4,27,1430,32.0,26.0,Scavenged,MBM + DBM,...,0.089623,0.470256,1.205369,0.014793,0.239572,0.281999,0.173245,0.215138,0.348744,0.384665
1,NB00003,Y,NB00003_M_1,29.0,22,995,35.7,23.7,Scavenged,DBM,...,0.054806,0.337448,0.352980,0.011570,0.075870,0.375191,0.066067,0.044498,0.038670,0.034095
2,NB00003,Y,NB00003_M_2,29.0,22,1005,36.0,24.4,Scavenged,DBM,...,0.113728,0.508929,0.384073,0.009342,0.082319,0.296118,0.063142,0.073261,0.057761,0.057419
3,NB00003,Y,NB00003_M_3,29.3,25,1065,36.0,24.3,Scavenged,DBM,...,0.078861,0.579639,0.381617,0.012011,0.080428,0.181844,0.061159,0.052973,0.090722,0.052794
4,NB00354,N,NB00354_M_1,29.4,12,1005,34.0,34.5,Scavenged,MBM,...,0.049258,0.108794,1.039603,0.024323,0.258750,0.138556,0.164200,0.203099,0.342770,0.364438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,NB00007,Y,NB00007_M_3,39.2,17,2340,44.0,33.0,Scavenged,MBM,...,0.093871,0.826393,0.565923,0.023937,0.160066,0.732057,0.248158,0.140775,0.036514,0.117773
131,NB00007,Y,NB00007_M_4,39.3,18,2380,44.0,33.0,Scavenged,MBM,...,0.062626,0.250724,1.749928,0.062896,0.362996,0.159926,0.227344,0.067247,0.204275,0.226578
132,NB00379,Y,NB00379_M_1,39.6,98,3073,45.0,33.2,Scavenged,MBM,...,0.074441,1.492769,0.211561,0.009555,0.160130,0.005741,0.066606,0.061999,0.013559,0.146605
133,NB00007,Y,NB00007_M_5,40.0,23,2430,45.0,34.5,Scavenged,MBM,...,0.101503,0.756176,0.861343,0.024187,0.165676,0.767132,0.270879,0.059874,0.098085,0.144319


In [115]:
#list of IDs in the metadata but not the AC file

set(df['sample_unique_id']) - set(AC['sample_unique_id'])


{'NB00001_M_8',
 'NB00002_M_7',
 'NB00003_M_24',
 'NB00003_M_25',
 'NB00003_M_5',
 'NB00007_M_7',
 'NB00423_M_1'}

In [116]:
#list of IDs in the AC file but not in the metadata

set(AC['sample_unique_id']) - set(df['sample_unique_id'])


{'NB00001_M_2',
 'NB00002_M_9 ?',
 'NB00007_M_6',
 'NB00008_M_6',
 'NB00101_M_1',
 'NB00361_M_6',
 'NB00361_M_7'}

In [117]:
merged["Scavenged/Fresh?"].value_counts()

Scavenged/Fresh?
Scavenged     109
Scavenged      18
scavenged       1
Name: count, dtype: int64

In [118]:
def normalize_scavenged(value):
    if isinstance(value, str) and value.strip().lower().startswith('scavenged'):
        return 'Scavenged'
    return value

merged["Scavenged/Fresh?"] = merged["Scavenged/Fresh?"].apply(normalize_scavenged)

In [119]:
merged["Scavenged/Fresh?"].value_counts()

Scavenged/Fresh?
Scavenged    128
Name: count, dtype: int64

In [120]:
def code_mbm(value):
    if isinstance(value, str) and value.strip().lower().replace(' ', '') == 'mbm':
        return 'MBM'
    return value

merged["MBM/DMB?"] = merged["MBM/DMB?"].apply(code_mbm)

In [121]:
merged["MBM/DMB?"].value_counts()

MBM/DMB?
MBM                       92
DBM                       21
Switched to Fortifier      9
MBM + DBM                  2
FBM/MBM                    1
Name: count, dtype: int64

In [122]:
merged = merged.rename(columns={"Iron Y/N?\xa0": "Iron Y/N"})

In [123]:
merged["Iron Y/N"].value_counts()

Iron Y/N
Y    48
N    25
Name: count, dtype: int64

In [124]:
import re

def extract_numeric_aliquots(value):
    match = re.match(r'^\d+', str(value))
    return int(match.group()) if match else None

merged['Aliquots_num'] = merged['# Aliquots '].apply(extract_numeric_aliquots)

In [125]:
merged["Aliquots_num"].value_counts()

Aliquots_num
1     54
7     12
5     10
2      9
3      9
9      8
4      7
8      6
6      6
10     3
13     2
14     1
11     1
Name: count, dtype: int64

In [126]:
merged = merged.rename(columns={"2'FL": "2FL", "3'SL": "3SL", "6'SL": "6SL"})

In [127]:
merged["2FL"].describe()

count    126.000000
mean       2.668561
std        2.210266
min        0.034985
25%        0.300394
50%        2.904018
75%        4.311703
max        8.572130
Name: 2FL, dtype: float64

In [128]:
import plotly.express as px
fig = px.histogram(merged, x="2FL", nbins=40, title='2fl Concentration Distribution')
fig

In [129]:
# Classify secretor status using 5M AUC cutoff
merged["Secretor Status"] = merged["2FL"].apply(
    lambda x: "Secretor" if x >= .5 else "Non-Secretor"
)

In [130]:
merged["Secretor Status"].value_counts()

Secretor Status
Secretor        81
Non-Secretor    47
Name: count, dtype: int64

In [131]:
merged.columns

Index(['Subject ID', 'Patient Consented', 'sample_unique_id', 'CGA', 'DOL ',
       'Current Weight', 'Current Height', 'Current HC', 'Scavenged/Fresh?',
       'MBM/DMB?', 'HMF Y/N?', 'TPN Y/N?', 'Iron Y/N', 'Linked? ',
       '# Aliquots ', 'Additional Comments', 'inj vol', '2FL', 'DFLAC', '3SL',
       '6SL', 'LNT', 'LNnT', 'LNFPI', 'LNFPII', 'LNFPIII', 'LSTc', 'DFLNT',
       'DSLNT', 'DFLNH', 'FDSLNH', 'DSLNH', 'Aliquots_num', 'Secretor Status'],
      dtype='object')

In [132]:
merged["Additional Comments"].value_counts()

Additional Comments
Residual from Milk Prep Room                                                       68
Scavenged Feeding Tube                                                             48
Scavenged Feeding Tube + NaCl Oral Solution                                         2
Residual from Milk Prep Room & B2 Full                                              1
Scavenged Feeding Tube - Nurse needed to feed extra 1 mL                            1
Scavenged Feeding Tube - also started on sodium chloride oral solution 6/15/24      1
Scanvenged Feeding Tube - Collected Outside of 4 hr Window                          1
Scavenged Feeding Tube + NaCl Oral Solution - Pt intubated 6/25 & is NPO            1
Scavenged Feeding Tube - Outside 4-hr .indo. by 2 hours                             1
Scavenged Feeding Tube - Intubated 6/25 - NPO, restarted NG feeds 7/2               1
Scavenged Feeding Tube - Parenteral Nutrition Stopped                               1
Scavenged Feeding Tube - Last NG F

In [133]:
import re

def extract_scavenged_notes(comment):
    if isinstance(comment, str) and "scavenged feeding tube" in comment.lower():
        # Look for + or - and any text after it
        match = re.search(r"(scavenged feeding tube.*?)([+-].*)", comment, re.IGNORECASE)
        if match:
            return match.group(2).strip()
    return None

merged["scavenged notes"] = merged["Additional Comments"].apply(extract_scavenged_notes)

In [134]:
import re

def clean_scavenged_feeding_tube(comment):
    if isinstance(comment, str):
        # Find 'scavenged feeding tube' and remove everything after it (including +, -, or any text)
        match = re.search(r"(scavenged feeding tube)", comment, re.IGNORECASE)
        if match:
            return comment[:match.end()].strip()
    return comment

merged["Additional Comments"] = merged["Additional Comments"].apply(clean_scavenged_feeding_tube)

In [135]:
merged[["Additional Comments", "scavenged notes"]]

,Additional Comments,scavenged notes
0,Residual from Milk Prep Room & B2 Full,None
1,Scavenged Feeding Tube,None
2,Scavenged Feeding Tube,None
3,Scavenged Feeding Tube,None
4,Residual from Milk Prep Room,None
...,...,...
123,Residual from Milk Prep Room,None
124,Residual from Milk Prep Room,None
125,Residual from Milk Prep Room,None
126,Residual from Milk Prep Room - Baby not on NG ...,None


In [136]:
# Remove leading '+', '-', or spaces from 'scavenged notes'
merged["scavenged notes"] = merged["scavenged notes"].str.lstrip('+- ').replace('', None)

In [137]:
merged["scavenged notes"].value_counts()

scavenged notes
NaCl Oral Solution                                       2
Nurse needed to feed extra 1 mL                          1
also started on sodium chloride oral solution 6/15/24    1
NaCl Oral Solution - Pt intubated 6/25 & is NPO          1
Outside 4-hr .indo. by 2 hours                           1
Intubated 6/25 - NPO, restarted NG feeds 7/2             1
Parenteral Nutrition Stopped                             1
Last NG Feed 7/2                                         1
Name: count, dtype: int64

In [138]:
merged["Additional Comments"].value_counts()

Additional Comments
Residual from Milk Prep Room                                            68
Scavenged Feeding Tube                                                  57
Residual from Milk Prep Room & B2 Full                                   1
Scanvenged Feeding Tube - Collected Outside of 4 hr Window               1
Residual from Milk Prep Room - Baby not on NG Feeds no linked sample     1
Name: count, dtype: int64

In [139]:
def classify_sample_method(comment):
    if isinstance(comment, str):
        if "scavenged" in comment.lower():
            return "Scavenged"
        elif "milk prep room" in comment.lower():
            return "Residual"
    return "Scavenged"

merged["Sample Source"] = merged["Additional Comments"].apply(classify_sample_method)


In [140]:
merged["Sample Source"].value_counts()

Sample Source
Residual     70
Scavenged    58
Name: count, dtype: int64

In [141]:
merged["CGA_cat"] = pd.cut(
    df["CGA"],
    bins=[0, 32, 34, 36, 45],
    labels=["Very Preterm", "Moderate Preterm", "Late Preterm", "Term"],
    include_lowest=True
)

In [142]:
merged["CGA_cat"].value_counts()

CGA_cat
Late Preterm        37
Very Preterm        36
Moderate Preterm    32
Term                23
Name: count, dtype: int64

#### download excel!

In [143]:
merged.to_excel('Unlinked_Merged.xlsx', index=False)

In [144]:
merged

,Subject ID,Patient Consented,sample_unique_id,CGA,DOL,Current Weight,Current Height,Current HC,Scavenged/Fresh?,MBM/DMB?,...,DFLNT,DSLNT,DFLNH,FDSLNH,DSLNH,Aliquots_num,Secretor Status,scavenged notes,Sample Source,CGA_cat
0,NB00012,N,NB00012_M_1,28.4,27,1430,32.0,26.0,Scavenged,MBM + DBM,...,0.281999,0.173245,0.215138,0.348744,0.384665,5,Secretor,None,Residual,Very Preterm
1,NB00003,Y,NB00003_M_1,29.0,22,995,35.7,23.7,Scavenged,DBM,...,0.375191,0.066067,0.044498,0.038670,0.034095,2,Secretor,None,Scavenged,Very Preterm
2,NB00003,Y,NB00003_M_2,29.0,22,1005,36.0,24.4,Scavenged,DBM,...,0.296118,0.063142,0.073261,0.057761,0.057419,1,Secretor,None,Scavenged,Very Preterm
3,NB00003,Y,NB00003_M_3,29.3,25,1065,36.0,24.3,Scavenged,DBM,...,0.181844,0.061159,0.052973,0.090722,0.052794,1,Secretor,None,Scavenged,Very Preterm
4,NB00354,N,NB00354_M_1,29.4,12,1005,34.0,34.5,Scavenged,MBM,...,0.138556,0.164200,0.203099,0.342770,0.364438,7,Non-Secretor,None,Residual,Very Preterm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,NB00007,Y,NB00007_M_2,38.4,12,2190,44.0,32.5,Scavenged,MBM,...,0.681753,0.194094,0.063247,0.092453,0.154003,7,Secretor,None,Residual,Term
124,NB00007,Y,NB00007_M_3,39.2,17,2340,44.0,33.0,Scavenged,MBM,...,0.732057,0.248158,0.140775,0.036514,0.117773,6,Secretor,None,Residual,Term
125,NB00007,Y,NB00007_M_4,39.3,18,2380,44.0,33.0,Scavenged,MBM,...,0.159926,0.227344,0.067247,0.204275,0.226578,7,Non-Secretor,None,Residual,Term
126,NB00379,Y,NB00379_M_1,39.6,98,3073,45.0,33.2,Scavenged,MBM,...,0.005741,0.066606,0.061999,0.013559,0.146605,7,Secretor,None,Residual,Term


### Create HMO SUM merged df

In [145]:
copy = merged.copy()

In [146]:
hmo_columns = [
    "2FL", "DFLAC", "3SL", "6SL", "LNT", "LNnT", "LNFPI", "LNFPII", "LNFPIII",
    "LSTc", "DFLNT", "DSLNT", "DFLNH", "FDSLNH", "DSLNH"
]

copy = copy.drop(columns=hmo_columns)

In [147]:
copy

,Subject ID,Patient Consented,sample_unique_id,CGA,DOL,Current Weight,Current Height,Current HC,Scavenged/Fresh?,MBM/DMB?,...,Iron Y/N,Linked?,# Aliquots,Additional Comments,inj vol,Aliquots_num,Secretor Status,scavenged notes,Sample Source,CGA_cat
0,NB00012,N,NB00012_M_1,28.4,27,1430,32.0,26.0,Scavenged,MBM + DBM,...,NaN,N,5 (~0.8),Residual from Milk Prep Room & B2 Full,15,5,Secretor,None,Residual,Very Preterm
1,NB00003,Y,NB00003_M_1,29.0,22,995,35.7,23.7,Scavenged,DBM,...,Y,N,2,Scavenged Feeding Tube,15,2,Secretor,None,Scavenged,Very Preterm
2,NB00003,Y,NB00003_M_2,29.0,22,1005,36.0,24.4,Scavenged,DBM,...,Y,N,1,Scavenged Feeding Tube,15,1,Secretor,None,Scavenged,Very Preterm
3,NB00003,Y,NB00003_M_3,29.3,25,1065,36.0,24.3,Scavenged,DBM,...,Y,N,1,Scavenged Feeding Tube,15,1,Secretor,None,Scavenged,Very Preterm
4,NB00354,N,NB00354_M_1,29.4,12,1005,34.0,34.5,Scavenged,MBM,...,NaN,N,7,Residual from Milk Prep Room,15,7,Non-Secretor,None,Residual,Very Preterm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,NB00007,Y,NB00007_M_2,38.4,12,2190,44.0,32.5,Scavenged,MBM,...,NaN,N,7,Residual from Milk Prep Room,15,7,Secretor,None,Residual,Term
124,NB00007,Y,NB00007_M_3,39.2,17,2340,44.0,33.0,Scavenged,MBM,...,NaN,N,6 (~0.8),Residual from Milk Prep Room,15,6,Secretor,None,Residual,Term
125,NB00007,Y,NB00007_M_4,39.3,18,2380,44.0,33.0,Scavenged,MBM,...,NaN,N,7,Residual from Milk Prep Room,15,7,Non-Secretor,None,Residual,Term
126,NB00379,Y,NB00379_M_1,39.6,98,3073,45.0,33.2,Scavenged,MBM,...,NaN,N,7,Residual from Milk Prep Room - Baby not on NG ...,15,7,Secretor,None,Residual,Term


In [148]:
AC_SUM = pd.read_excel('Data/NeoBank Unlinked AC.xlsx', sheet_name='raw')
AC_SUM.head()

,sample ID,2'FL,Iso (control),DFLAC,3'SL,6'SL,LNT,LNnT,LNFPI,LNFPII,LNFPIII,LSTc,DFLNT,DSLNT,DFLNH,FDSLNH,DSLNH
0,NB00003_M_27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NB00003_M_26,1.051478e+06,7053701.787,1.801466e+05,2.556277e+05,3.403535e+05,2.309651e+05,1.091730e+05,2.635778e+05,2.057756e+05,101484.2646,2.069264e+05,1.724065e+05,1.696447e+05,99897.0777,172129.7542,1.006923e+05
2,NB00010_M_4,2.224422e+07,6946214.190,1.466091e+06,1.403590e+06,5.509796e+06,8.950674e+06,6.919087e+05,6.229519e+06,5.821316e+06,182680.4001,1.378025e+06,5.670600e+06,2.037842e+06,449381.8137,810615.6873,1.103405e+06
3,NB00004_M_4,3.213107e+07,7087612.145,1.108820e+05,7.738709e+05,3.660383e+06,6.932930e+06,1.667270e+06,1.513378e+07,1.778976e+05,199407.6536,1.938777e+06,4.739951e+05,1.129592e+06,409684.5743,233999.6664,4.959159e+05
4,NB00014_M_1,4.736746e+07,7159181.787,7.137924e+06,2.049916e+06,5.323517e+06,7.671713e+06,2.415186e+06,1.587956e+07,2.630392e+06,365193.0543,4.721838e+06,5.694380e+06,1.872350e+06,274316.2401,345379.6165,1.061841e+06


In [149]:
AC_SUM = AC_SUM.rename(columns={"sample ID": "sample_unique_id"})
AC_SUM

,sample_unique_id,2'FL,Iso (control),DFLAC,3'SL,6'SL,LNT,LNnT,LNFPI,LNFPII,LNFPIII,LSTc,DFLNT,DSLNT,DFLNH,FDSLNH,DSLNH
0,NB00003_M_27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NB00003_M_26,1.051478e+06,7.053702e+06,1.801466e+05,2.556277e+05,3.403535e+05,2.309651e+05,1.091730e+05,2.635778e+05,2.057756e+05,1.014843e+05,2.069264e+05,1.724065e+05,1.696447e+05,99897.0777,1.721298e+05,1.006923e+05
2,NB00010_M_4,2.224422e+07,6.946214e+06,1.466091e+06,1.403590e+06,5.509796e+06,8.950674e+06,6.919087e+05,6.229519e+06,5.821316e+06,1.826804e+05,1.378025e+06,5.670600e+06,2.037842e+06,449381.8137,8.106157e+05,1.103405e+06
3,NB00004_M_4,3.213107e+07,7.087612e+06,1.108820e+05,7.738709e+05,3.660383e+06,6.932930e+06,1.667270e+06,1.513378e+07,1.778976e+05,1.994077e+05,1.938777e+06,4.739951e+05,1.129592e+06,409684.5743,2.339997e+05,4.959159e+05
4,NB00014_M_1,4.736746e+07,7.159182e+06,7.137924e+06,2.049916e+06,5.323517e+06,7.671713e+06,2.415186e+06,1.587956e+07,2.630392e+06,3.651931e+05,4.721838e+06,5.694380e+06,1.872350e+06,274316.2401,3.453796e+05,1.061841e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,NB00380_M_5,2.548159e+06,8.374666e+06,9.963675e+04,2.203344e+06,6.868806e+06,1.093594e+07,2.809632e+06,1.106507e+06,1.156049e+07,1.443196e+05,3.168754e+06,1.494014e+06,1.248896e+06,837453.6015,2.066399e+06,2.087714e+06
131,NB00438_M_3,3.714314e+07,8.849808e+06,3.128912e+06,1.859451e+06,4.517493e+06,7.412311e+06,1.563939e+06,1.331367e+07,3.463945e+06,1.708952e+05,2.663312e+06,5.194955e+06,9.190932e+05,387868.2138,3.366231e+05,1.031563e+06
132,NB00457_M_1,7.070979e+07,9.802165e+06,5.448618e+06,3.358550e+06,1.142248e+07,2.260011e+07,2.418345e+06,3.402642e+07,7.158418e+06,1.070328e+06,8.963838e+06,1.564165e+07,3.054897e+06,474998.2730,2.857115e+05,1.594684e+06
133,NB00468_M_1,3.233975e+07,9.937304e+06,3.918450e+06,2.446762e+06,3.289483e+06,4.012047e+06,9.774770e+05,4.947740e+06,5.697583e+06,1.941846e+05,1.292248e+06,5.087703e+06,9.968905e+05,662102.2885,6.792672e+05,6.872472e+05


In [150]:
merged_sum = pd.merge(copy, AC_SUM, on="sample_unique_id", how="inner")
merged_sum

,Subject ID,Patient Consented,sample_unique_id,CGA,DOL,Current Weight,Current Height,Current HC,Scavenged/Fresh?,MBM/DMB?,...,LNnT,LNFPI,LNFPII,LNFPIII,LSTc,DFLNT,DSLNT,DFLNH,FDSLNH,DSLNH
0,NB00012,N,NB00012_M_1,28.4,27,1430,32.0,26.0,Scavenged,MBM + DBM,...,7.026127e+05,3.686662e+06,9.449716e+06,115974.6422,1.878166e+06,2.210782e+06,1.358187e+06,1.686614e+06,2.734042e+06,3.015652e+06
1,NB00003,Y,NB00003_M_1,29.0,22,995,35.7,23.7,Scavenged,DBM,...,3.554526e+05,2.188585e+06,2.289321e+06,75037.6026,4.920676e+05,2.433375e+06,4.284901e+05,2.886016e+05,2.508040e+05,2.211292e+05
2,NB00003,Y,NB00003_M_2,29.0,22,1005,36.0,24.4,Scavenged,DBM,...,1.009692e+06,4.518359e+06,3.409862e+06,82939.5756,7.308408e+05,2.628986e+06,5.605861e+05,6.504227e+05,5.128080e+05,5.097773e+05
3,NB00003,Y,NB00003_M_3,29.3,25,1065,36.0,24.3,Scavenged,DBM,...,6.291601e+05,4.624398e+06,3.044566e+06,95822.1100,6.416626e+05,1.450763e+06,4.879326e+05,4.226243e+05,7.237869e+05,4.211949e+05
4,NB00354,N,NB00354_M_1,29.4,12,1005,34.0,34.5,Scavenged,MBM,...,3.386165e+05,7.478814e+05,7.146520e+06,167202.2557,1.778718e+06,9.524693e+05,1.128755e+06,1.396158e+06,2.356293e+06,2.505246e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,NB00007,Y,NB00007_M_2,38.4,12,2190,44.0,32.5,Scavenged,MBM,...,1.792379e+06,9.377929e+06,6.286603e+06,347147.3171,3.210144e+06,6.198172e+06,1.764609e+06,5.750102e+05,8.405371e+05,1.400118e+06
126,NB00007,Y,NB00007_M_3,39.2,17,2340,44.0,33.0,Scavenged,MBM,...,6.844752e+05,6.025784e+06,4.126527e+06,174538.3580,1.167148e+06,5.337920e+06,1.809488e+06,1.026485e+06,2.662468e+05,8.587586e+05
127,NB00007,Y,NB00007_M_4,39.3,18,2380,44.0,33.0,Scavenged,MBM,...,5.297594e+05,2.120913e+06,1.480291e+07,532044.6917,3.070643e+06,1.352836e+06,1.923140e+06,5.688529e+05,1.727998e+06,1.916662e+06
128,NB00379,Y,NB00379_M_1,39.6,98,3073,45.0,33.2,Scavenged,MBM,...,6.555827e+05,1.314651e+07,1.863176e+06,84150.5867,1.410230e+06,5.056181e+04,5.865819e+05,5.460105e+05,1.194098e+05,1.291120e+06


In [151]:
# List of HMO columns in merged_sum, excluding 'Iso (control)'
hmo_sum_columns = [
    "2'FL", "DFLAC", "3'SL", "6'SL", "LNT", "LNnT", "LNFPI", "LNFPII", "LNFPIII",
    "LSTc", "DFLNT", "DSLNT", "DFLNH", "FDSLNH", "DSLNH"
]

# Sum HMOs for each row and create a new column
merged_sum['HMO_sum'] = merged_sum[hmo_sum_columns].sum(axis=1)

In [152]:
merged_sum.columns = [col.replace("'", "") for col in merged_sum.columns]

In [153]:
# Choose a cutoff for 2FL (adjust based on your AUC distribution)
merged_sum["Secretor Status"] = merged_sum["2FL"].apply(
    lambda x: "Secretor" if x >= 5_000_000 else "Non-Secretor"
)

In [155]:
merged_sum["Secretor Status"].value_counts()

Secretor Status
Secretor        80
Non-Secretor    50
Name: count, dtype: int64

In [ ]:
merged_sum

In [156]:
merged_sum.to_excel('Unlinked_HMO_SUMS.xlsx', index=False)

### is HMO concentration accosicated with metadata features?

In [ ]:
pip install scipy


In [ ]:
merged

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu

sns.boxplot(data=merged, x="TPN Y/N?", y="2FL")
plt.title("2’FL Concentration by TPN Use")
plt.show()

# Mann-Whitney U test (non-parametric)
group1 = merged[merged["TPN Y/N?"] == "Y"]["2FL"].dropna()
group2 = merged[merged["TPN Y/N?"] == "N"]["2FL"].dropna()

stat, p = mannwhitneyu(group1, group2)
print(f"Mann-Whitney U test: U={stat:.2f}, p={p:.4f}")


### Only with Subject IDs with more than 3 timepoints

In [ ]:
subject_counts = merged["Subject ID"].value_counts()
longitudinal_subjects = subject_counts[subject_counts > 3].index

filtered_df = merged[merged["Subject ID"].isin(longitudinal_subjects)].copy()

In [ ]:
filtered_df.columns = filtered_df.columns.str.strip()


In [ ]:
filtered_df["MBM/DMB?"]

In [ ]:
filtered_df.columns

In [ ]:
df = merged.rename(columns=lambda x: x.strip())  # Remove trailing spaces
df["DOL"] = pd.to_numeric(df["DOL"], errors="coerce")

In [ ]:
import plotly.express as px

fig = px.line(
    df[df["Subject ID"] == 'NB00003'],  # set selected_subject as needed
    x="DOL",
    y="2FL",
    color="TPN Y/N?",  # or "TPN Y/N?"
    markers=True,
    title=f"2’FL over Time for {'NB00003'} by HMF Status"
)
fig.update_layout(
    plot_bgcolor="#1E1E1E",
    paper_bgcolor="#1E1E1E",
    font_color="white"
)
fig.show()


In [ ]:
from scipy.stats import mannwhitneyu

group1 = df[df["TPN Y/N?"] == "Y"]["2FL"].dropna()
group2 = df[df["TPN Y/N?"] == "N"]["2FL"].dropna()

stat, p = mannwhitneyu(group1, group2)
print(f"TPN Effect on 2’FL: U = {stat:.2f}, p = {p:.4f}")


In [ ]:
df.columns

In [ ]:
df["hover_label"] = (
    "TPN: " + df["TPN Y/N?"].astype(str) + "<br>" +
    "HMF: " + df["HMF Y/N?"].astype(str) + "<br>" +
    "Iron: " + df["Iron Y/N"].astype(str) + "<br>" +
    "Scavenged: " + df["Scavenged/Fresh?"].astype(str) + "<br>" +
    "Notes: " + df["Additional Comments"].astype(str)
)

In [ ]:
import plotly.express as px

In [ ]:
df["Iron Size"] = df["Iron Y/N"].map({"Y": 20, "N": 10})


In [ ]:
import plotly.express as px

fig = px.scatter(
    df[df["Subject ID"] == "NB00003"],
    x="DOL",
    y="2FL",
    color="TPN Y/N?",
    symbol="Scavenged/Fresh?",
    size="Iron Size",
    hover_name="DOL",
    hover_data={
        "TPN Y/N?": True,
        "HMF Y/N?": True,
        "Iron Y/N": True,
        "Scavenged/Fresh?": True,
        "Additional Comments": True
    },
    text="HMF Y/N?",
    title="2’FL over Time for NB00003"
)


fig.update_layout(
    plot_bgcolor="#1E1E1E",
    paper_bgcolor="#1E1E1E",
    font_color="white"
)

fig.show()


## Type of Milk Collected

In [ ]:
merged["Additional Comments"].value_counts()

In [ ]:
filtered_merged = merged[
    merged["Additional Comments"].isin(["Residual from Milk Prep Room", "Scavenged Feeding Tube"])
].copy()

In [ ]:
filtered_merged

In [ ]:
def classify_sample_method(comment):
    if isinstance(comment, str):
        if "scavenged feeding tube" in comment.lower():
            return "Scavenged"
        elif "milk prep room" in comment.lower():
            return "Residual"
    return "Other"

filtered_merged["Sample Source"] = filtered_merged["Additional Comments"].apply(classify_sample_method)


In [ ]:
filtered_merged["Sample Source"].value_counts()

In [ ]:
filtered_merged["CGA"].describe()

In [ ]:
filtered_merged["CGA Bin"] = pd.cut(
    filtered_merged["CGA"], 
    bins=[28, 32, 34, 36, 41], 
    labels=["Very Preterm (<32)", "Moderate Preterm (32-34)", "Late Preterm (34–36)", "Term (35–40)"]
)


In [ ]:
filtered_merged[""]